<a href="https://colab.research.google.com/github/apschlissel/w266-final-project/blob/main/BERT_reddit_baseline_(Kevin).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [32]:
# !config --NotebookApp.iopub_data_rate_limit=1.0e10
# # Source: https://stackoverflow.com/questions/50689784/in-google-collab-i-get-iopub-data-rate-exceeded

/bin/bash: config: command not found


In [1]:
!pip install -q transformers

     |████████████████████████████████| 3.8 MB 4.7 MB/s 
     |████████████████████████████████| 67 kB 2.6 MB/s 
     |████████████████████████████████| 6.5 MB 12.4 MB/s 
     |████████████████████████████████| 596 kB 32.0 MB/s 
     |████████████████████████████████| 895 kB 34.6 MB/s 


In [2]:
!pip install simpletransformers

     |████████████████████████████████| 248 kB 5.3 MB/s 
     |████████████████████████████████| 9.9 MB 22.3 MB/s 
     |████████████████████████████████| 1.2 MB 33.2 MB/s 
     |████████████████████████████████| 1.7 MB 30.6 MB/s 
     |████████████████████████████████| 43 kB 1.6 MB/s 
     |████████████████████████████████| 325 kB 14.7 MB/s 
     |████████████████████████████████| 144 kB 8.3 MB/s 
     |████████████████████████████████| 181 kB 31.1 MB/s 
     |████████████████████████████████| 63 kB 542 kB/s 
     |████████████████████████████████| 212 kB 9.9 MB/s 
     |████████████████████████████████| 134 kB 10.8 MB/s 
     |████████████████████████████████| 1.1 MB 26.2 MB/s 
     |████████████████████████████████| 127 kB 14.9 MB/s 
     |████████████████████████████████| 94 kB 1.5 MB/s 
     |████████████████████████████████| 271 kB 35.4 MB/s 
     |████████████████████████████████| 144 kB 32.6 MB/s 
     |████████████████████████████████| 4.3 MB 18.1 MB/s 
     |█████████████████

In [3]:
# Useful Links / Source Code
# Source: https://towardsdatascience.com/multi-class-text-classification-with-deep-learning-using-bert-b59ca2f5c613
# Source: https://github.com/susanli2016/NLP-with-Python/blob/master/Text_Classification_With_BERT.ipynb
# Source: https://towardsdatascience.com/how-to-use-the-reddit-api-in-python-5e05ddfd1e5c
# Source: https://medium.com/@senthilnathangautham/colab-gcp-compute-how-to-link-them-together-98747e8d940e
# Source: https://stackoverflow.com/questions/49031798/when-i-use-google-colaboratory-how-to-save-image-weights-in-my-google-drive

In [2]:
from __future__ import print_function
import ipywidgets as widgets
from transformers import pipeline
from simpletransformers.t5 import T5Model, T5Args
import pandas as pd
import logging
import numpy as np
import torch
from tqdm.notebook import tqdm
from transformers import BertTokenizer
from torch.utils.data import TensorDataset
from transformers import BertForSequenceClassification
import json
# Pull reddit data from reddit api
import requests

In [5]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
# note that CLIENT_ID refers to 'personal use script' and SECRET_TOKEN to 'token'
auth = requests.auth.HTTPBasicAuth('pigKA_TKnDkXcatEGcbo8g', 'nawGKK2MfPtC6vKz8TjaNEnmYfAggA')

# here we pass our login method (password), username, and password
data = {'grant_type': 'password',
        'username': 'Katsuuu100',
        'password': 'Testing159753'}

# setup our header info, which gives reddit a brief description of our app
headers = {'User-Agent': 'MyBot/0.0.1'}

# send our request for an OAuth token
res = requests.post('https://www.reddit.com/api/v1/access_token',
                    auth=auth, data=data, headers=headers)

# convert response to JSON and pull access_token value
TOKEN = res.json()['access_token']

# add authorization to our headers dictionary
headers = {**headers, **{'Authorization': f"bearer {TOKEN}"}}

# while the token is valid (~2 hours) we just add headers=headers to our requests
requests.get('https://oauth.reddit.com/api/v1/me', headers=headers)

<Response [200]>

In [4]:
# Source: https://towardsdatascience.com/how-to-use-the-reddit-api-in-python-5e05ddfd1e5c

# Pull from 5 classes. 5 classes = 5 subreddits.

# url_list = ["https://oauth.reddit.com/r/wallstreetbets/new",
            # "https://oauth.reddit.com/r/teenagers/new",
            # "https://oauth.reddit.com/r/copypasta/new",
              # "https://oauth.reddit.com/r/genz/new",
              # "https://oauth.reddit.com/r/unpopularopinion/new"]

my_list_of_dictionaries = []
total = 0


for i in range(5):
  page_count = 25
  url_list_check = [f"https://oauth.reddit.com/r/wallstreetbets/new/",
          f"https://oauth.reddit.com/r/teenagers/new/",
          f"https://oauth.reddit.com/r/copypasta/new/",
          f"https://oauth.reddit.com/r/genz/new/",
          f"https://oauth.reddit.com/r/unpopularopinion/new/",
          ]
  
  #print(url_list_check[i])
  if "wallstreetbets" in url_list_check[i]:
    name = "t3_tjoebd"
  elif "teenagers" in url_list_check[i]:
    name = "t3_tjt2rv"
  elif "copypasta" in url_list_check[i]:
    name = "t3_tjpvmw"
  elif "genz" in url_list_check[i]:
    name = "t3_ti9ds3"
  elif "unpopularopinion" in url_list_check[i]:
    name = "t3_tjpqel"
  for j in range(4):

    url_list = [f"https://oauth.reddit.com/r/wallstreetbets/new/?count={page_count}&after={name}",
            f"https://oauth.reddit.com/r/teenagers/new/?count={page_count}&after={name}",
            f"https://oauth.reddit.com/r/copypasta/new/?count={page_count}",
            f"https://oauth.reddit.com/r/genz/new/?count={page_count}",
            f"https://oauth.reddit.com/r/unpopularopinion/new/?count={page_count}",
            ]
    print(page_count)
    print(url_list[i])
    res = requests.get(url_list[i],
                    headers=headers)
                    # params={"limit": "100"})

    #print(json.dumps(res.json(), indent=4, sort_keys=False))

    reddit_dictionary = res.json()
    # print(json.dumps(reddit_dictionary["data"]["children"][0], indent=4, sort_keys=False))
    # print(len(reddit_dictionary["data"]["children"]))
    for k in range(len(reddit_dictionary["data"]["children"])):
      my_dictionary = {}
      my_dictionary["subreddit"] = reddit_dictionary["data"]["children"][k]["data"]["subreddit"]
      my_dictionary["text"] = reddit_dictionary["data"]["children"][k]["data"]["selftext"]
      my_dictionary["title"] = reddit_dictionary["data"]["children"][k]["data"]["title"]
      my_dictionary["url"] = reddit_dictionary["data"]["children"][k]["data"]["url"]
      # print(reddit_dictionary["data"]["children"][k]["data"]["subreddit"])
      # print(reddit_dictionary["data"]["children"][k]["data"]["selftext"])
      # print(reddit_dictionary["data"]["children"][k]["data"]["url"])
      my_list_of_dictionaries.append(my_dictionary)
      total += 1
      name = reddit_dictionary["data"]["children"][k]["data"]["name"]
    # print(json.dumps(my_list_of_dictionaries, indent=4, sort_keys=False))
    print(name)
    page_count += 25

print("Total gathered:", total)

25
https://oauth.reddit.com/r/wallstreetbets/new/?count=25&after=t3_tjoebd
t3_tj5sx0
50
https://oauth.reddit.com/r/wallstreetbets/new/?count=50&after=t3_tj5sx0
t3_tia7aj
75
https://oauth.reddit.com/r/wallstreetbets/new/?count=75&after=t3_tia7aj
t3_ti0px1
100
https://oauth.reddit.com/r/wallstreetbets/new/?count=100&after=t3_ti0px1
t3_thvmxt
25
https://oauth.reddit.com/r/teenagers/new/?count=25&after=t3_tjt2rv
t3_tjswvf
50
https://oauth.reddit.com/r/teenagers/new/?count=50&after=t3_tjswvf
t3_tjssh4
75
https://oauth.reddit.com/r/teenagers/new/?count=75&after=t3_tjssh4
t3_tjsjbh
100
https://oauth.reddit.com/r/teenagers/new/?count=100&after=t3_tjsjbh
t3_tjsedn
25
https://oauth.reddit.com/r/copypasta/new/?count=25
t3_tjre4x
50
https://oauth.reddit.com/r/copypasta/new/?count=50
t3_tjre4x
75
https://oauth.reddit.com/r/copypasta/new/?count=75
t3_tjre4x
100
https://oauth.reddit.com/r/copypasta/new/?count=100
t3_tjre4x
25
https://oauth.reddit.com/r/genz/new/?count=25
t3_tialrr
50
https://oauth.re

In [5]:
# Source: https://github.com/susanli2016/NLP-with-Python/blob/master/Text_Classification_With_BERT.ipynb
# Convert list of dictionaries into pandas df
df = pd.DataFrame(my_list_of_dictionaries)
df.head()

,subreddit,text,title,url
0,wallstreetbets,,Accepting a share buyback without any shares =...,https://i.redd.it/k10jqp0pgto81.jpg
1,wallstreetbets,China Evergrande Group and its other units wer...,Evergrande Trading Suspended in Hong Kong; Ron...,https://www.reddit.com/r/wallstreetbets/commen...
2,wallstreetbets,"Ten months ago, u/robertisaak proposed, [here]...",South Park Investment Theory (deeper dive),https://www.reddit.com/r/wallstreetbets/commen...
3,wallstreetbets,"Checking flight radars, I have noticed a highl...",WSB AIR FORCE INTEL,https://www.reddit.com/r/wallstreetbets/commen...
4,wallstreetbets,,2 week 10x challenge. My starting position.,https://i.redd.it/ttwo0lw6pso81.jpg


In [6]:
df['subreddit'].value_counts()

GenZ                100
teenagers           100
wallstreetbets      100
unpopularopinion    100
copypasta           100
Name: subreddit, dtype: int64

In [7]:
possible_labels = df.subreddit.unique()

label_dict = {}
for index, possible_label in enumerate(possible_labels):
    label_dict[possible_label] = index
label_dict

{'wallstreetbets': 0,
 'teenagers': 1,
 'copypasta': 2,
 'GenZ': 3,
 'unpopularopinion': 4}

In [8]:
df['label'] = df.subreddit.replace(label_dict)

In [9]:
df.head()

,subreddit,text,title,url,label
0,wallstreetbets,,Accepting a share buyback without any shares =...,https://i.redd.it/k10jqp0pgto81.jpg,0
1,wallstreetbets,China Evergrande Group and its other units wer...,Evergrande Trading Suspended in Hong Kong; Ron...,https://www.reddit.com/r/wallstreetbets/commen...,0
2,wallstreetbets,"Ten months ago, u/robertisaak proposed, [here]...",South Park Investment Theory (deeper dive),https://www.reddit.com/r/wallstreetbets/commen...,0
3,wallstreetbets,"Checking flight radars, I have noticed a highl...",WSB AIR FORCE INTEL,https://www.reddit.com/r/wallstreetbets/commen...,0
4,wallstreetbets,,2 week 10x challenge. My starting position.,https://i.redd.it/ttwo0lw6pso81.jpg,0


In [14]:
#de-slang text
slang = pd.read_csv('slangit.csv')
slangit_dict = slang.set_index('Slang Term').to_dict()
slangit_dict = slangit_dict['Meaning']

def slang_lookup(text, dictionary):
    
    try:
        pattern = re.compile(r'(?<!\w)(' + '|'.join(re.escape(key) for key in dictionary.keys()) + r')(?!\w)')
        result = pattern.sub(lambda x: dictionary[x.group()], text)
    except:
        result = text

    return result

df['text_deslanged'] = df['text'].apply(lambda x: slang_lookup(x, slangit_dict))

df['text_deslanged'].head()

0                                                     
1    China Evergrande Group and its other units wer...
2    Ten months ago, u/robertisaak proposed, [here]...
3    Checking flight radars, I have noticed a highl...
4                                                     
Name: text_deslanged, dtype: object

In [15]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(df.index.values, 
                                                  df.label.values, 
                                                  test_size=0.15, 
                                                  random_state=42, 
                                                  stratify=df.label.values)

In [16]:
df['data_type'] = ['not_set']*df.shape[0]

df.loc[X_train, 'data_type'] = 'train'
df.loc[X_val, 'data_type'] = 'val'

In [17]:
df.groupby(['subreddit', 'label', 'data_type']).count()

text  title  url  text_deslanged
subreddit        label data_type                                  
GenZ             3     train        85     85   85              85
                       val          15     15   15              15
copypasta        2     train        85     85   85              85
                       val          15     15   15              15
teenagers        1     train        85     85   85              85
                       val          15     15   15              15
unpopularopinion 4     train        85     85   85              85
                       val          15     15   15              15
wallstreetbets   0     train        85     85   85              85
                       val          15     15   15              15

In [18]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', 
                                          do_lower_case=True)

In [19]:
encoded_data_train = tokenizer.batch_encode_plus(
    df[df.data_type=='train'].text_deslanged.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=256, 
    return_tensors='pt'
)

encoded_data_val = tokenizer.batch_encode_plus(
    df[df.data_type=='val'].text_deslanged.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=256, 
    return_tensors='pt'
)


input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(df[df.data_type=='train'].label.values)

input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(df[df.data_type=='val'].label.values)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/home/apschlissel/anaconda3/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:2271: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [20]:
dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)

In [21]:
len(dataset_train), len(dataset_val)

(425, 75)

In [22]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=len(label_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [23]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

batch_size = 3

dataloader_train = DataLoader(dataset_train, 
                              sampler=RandomSampler(dataset_train), 
                              batch_size=batch_size)

dataloader_validation = DataLoader(dataset_val, 
                                   sampler=SequentialSampler(dataset_val), 
                                   batch_size=batch_size)

In [24]:
from transformers import AdamW, get_linear_schedule_with_warmup

optimizer = AdamW(model.parameters(),
                  lr=1e-5, 
                  eps=1e-8)

/home/apschlissel/anaconda3/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [25]:
epochs = 5

scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps=0,
                                            num_training_steps=len(dataloader_train)*epochs)

In [26]:
from sklearn.metrics import f1_score

def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average='weighted')

def accuracy_per_class(preds, labels):
    label_dict_inverse = {v: k for k, v in label_dict.items()}
    
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        print(f'Class: {label_dict_inverse[label]}')
        print(f'Accuracy: {len(y_preds[y_preds==label])}/{len(y_true)}\n')

In [27]:
import random

seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [28]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

print(device)

cpu


In [29]:
def evaluate(dataloader_val):

    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in dataloader_val:
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals

In [ ]:
for epoch in tqdm(range(1, epochs+1)):
    
    model.train()
    
    loss_train_total = 0

    progress_bar = tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
    for batch in progress_bar:

        model.zero_grad()
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }       

        outputs = model(**inputs)
        
        loss = outputs[0]
        loss_train_total += loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()
        
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})
         
        
    torch.save(model.state_dict(), f'/content/gdrive/MyDrive/W266/finetuned_BERT_epoch_{epoch}.model')
        
    tqdm.write(f'\nEpoch {epoch}')
    
    loss_train_avg = loss_train_total/len(dataloader_train)            
    tqdm.write(f'Training loss: {loss_train_avg}')
    
    val_loss, predictions, true_vals = evaluate(dataloader_validation)
    val_f1 = f1_score_func(predictions, true_vals)
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 Score (Weighted): {val_f1}')

  0%|          | 0/5 [00:00<?, ?it/s]

Epoch 1:   0%|          | 0/142 [00:00<?, ?it/s]

In [ ]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=len(label_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False)

model.to(device)

In [ ]:
model.load_state_dict(torch.load('/content/gdrive/MyDrive/W266/finetuned_BERT_epoch_1.model', map_location=torch.device('cpu')))

In [ ]:
_, predictions, true_vals = evaluate(dataloader_validation)

In [ ]:
accuracy_per_class(predictions, true_vals)